<a href="https://colab.research.google.com/github/WhyVivek30/Project/blob/master/Use_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==2.3

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

In [34]:
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]



with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Message: Elephant
Embedding size: 512
Embedding: [0.04498472809791565, -0.057433921843767166, 0.0022114901803433895, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.055680178105831146, -0.009607940912246704, 0.006246272940188646, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.03874940052628517, 0.0765201598405838, -0.000794590450823307, ...]



In [35]:
# Compute a representation for each message, showing various lengths supported.
messages = ["That band rocks!", "That song is really cool."]

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))
message_embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[ 0.00341093, -0.0381117 , -0.01249461, ..., -0.03188571,
         0.03693528, -0.04350929],
       [ 0.00599076, -0.03070363,  0.02946817, ..., -0.01874218,
         0.05244366,  0.01933834]], dtype=float32)

In [36]:
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size

512

In [37]:
import codecs
import pandas as pd
def get_dataframe(filename):
    lines = open(filename, 'r', encoding="utf-8").read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(":")[0]
        text = ' '.join(lines[i].split(' ')[1:])
       # text = re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

df_train = get_dataframe('/content/drive/My Drive/Malayalam Task 2/malayalam_train.txt')
df_train.head()

,label,text
0,P,ਗੁਜਰਾਤ ਦੇ ਮੁੱਖ ਮੰਤਰੀ ਨਰਿੰਦਰ ਮੋਦੀ ਦਾ ਸਦਭਾਵਨਾ ਮਿ...
1,P,ਪੰਜਾਬ ਵਿਚ ਨਿੱਤ-ਦਿਨ ਵਾਪਰ ਰਹੀਆਂ ਲੁੱਟ-ਖਸੁੱਟ ਦੀਆਂ ...
2,P,ਪੱਛਮੀ ਬੰਗਾਲ ਵਿਚ 1977 ਤੋਂ ਲਗਾਤਾਰ ਖੱਬੇ-ਪੱਖੀ ਪਾਰਟ...
3,P,ਮਾਨਸਾ ਰੋਡ ‘ਤੇ ਸ੍ਰੀ ਵਾਲਮੀਕ ਚੌਕ ਨੇੜੇ ਤੇਲ ਟੈਂਕਰ ਨ...
4,P,ਪੁਲਿਸ ਨੇ ਢਕੌਲੀ ਦੇ ਆਦਰਸ਼ ਕਾਲੋਨੀ ਦੇ ਫਲੈਟ ਵਿਚ ਛਾਪ...


In [13]:
df_train = get_dataframe('/content/drive/My Drive/Malayalam Task 2/malayalam_train.txt')

In [14]:
category_counts = len(df_train.label.cat.categories)
category_counts

3

In [15]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [17]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 771       
Total params: 132,099
Trainable params: 132,099
Non-trainable params: 0
_________________________________________________________________


In [18]:
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [19]:
train_text.shape

(1759, 1)

In [20]:
train_label.shape

(1759, 3)

In [21]:
train_label[:3]

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=int8)

In [22]:
df_test = get_dataframe('/content/drive/My Drive/Malayalam Task 2/malayalam_dev.txt')

In [23]:
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [24]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=5,
            batch_size=32)
  model.save_weights('./model.h5')

Train on 1759 samples, validate on 441 samples
Epoch 1/5
1759/1759 [==============================] - 100s 57ms/step - loss: 1.0367 - accuracy: 0.4764 - val_loss: 1.1601 - val_accuracy: 0.3333
Epoch 2/5
1759/1759 [==============================] - 96s 55ms/step - loss: 1.0212 - accuracy: 0.4861 - val_loss: 1.1921 - val_accuracy: 0.3333
Epoch 3/5
1759/1759 [==============================] - 97s 55ms/step - loss: 1.0139 - accuracy: 0.4895 - val_loss: 1.1440 - val_accuracy: 0.3288
Epoch 4/5
1759/1759 [==============================] - 97s 55ms/step - loss: 1.0012 - accuracy: 0.4895 - val_loss: 1.1504 - val_accuracy: 0.3197
Epoch 5/5
1759/1759 [==============================] - 96s 55ms/step - loss: 0.9961 - accuracy: 0.4901 - val_loss: 1.1286 - val_accuracy: 0.3175


In [25]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root 531K Sep 21 14:09 model.h5


In [26]:
df_test = get_dataframe('/content/drive/My Drive/Malayalam Task 2/malayalam_test.txt')
new_text = df_test['text'].tolist()
new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)
  categories = df_train.label.cat.categories.tolist()
  predict_logits = predicts.argmax(axis=1)
  predict_labels = [categories[logit] for logit in predict_logits]
  predict_labels

In [27]:
with open('/content/drive/My Drive/Malayalam Task 2/malayalam_test_label_obtained.txt', 'w') as f:
    for item in predict_labels:
        f.write("%s\n" % item)

In [28]:
with open('/content/drive/My Drive/Malayalam Task 2/malayalam_probability.txt', 'w') as f:
    for item in predicts:
        f.write("%s\n" % item)

In [29]:
content_array_1 = []
content_array_2 = []
def file_read(fname,fname1):
        with open(fname) as f:
                #Content_list is the list that contains the read lines.     
                for line in f:
                        content_array_1.append(line)
                print(content_array_1)
        with open(fname1) as f:
                #Content_list is the list that contains the read lines.     
                for line in f:
                        content_array_2.append(line)
                print(content_array_2)        

file_read('/content/drive/My Drive/Malayalam Task 2/malayalam_test_label_obtained.txt','/content/drive/My Drive/Malayalam Task 2/malayalam_test_labels.txt')

['P\n', 'P\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'P\n', 'P\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'P\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'NP\n', 'N

In [30]:
from sklearn.metrics import accuracy_score
y_pred = content_array_1
y_true = content_array_2
accuracy_score(y_true, y_pred)
accuracy_score(y_true, y_pred, normalize=True)

0.3657142857142857

In [31]:
from sklearn.metrics import precision_score
y_true = content_array_2
y_pred = content_array_1
precision_score(y_true, y_pred, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2241810000420539

In [32]:
from sklearn.metrics import f1_score
y_true = content_array_2
y_pred = content_array_1
f1_score(y_true, y_pred, average='weighted')

0.25139769159455055